In [4]:
# Take the V1 example and use for inference in the app. Testing notebook

In [5]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
def make_recommendation(gamelist_length, game):
    # Step 1: Load the dataset
    df = pd.read_csv("resources/ml_clean.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # Drop unnecessary columns that are not needed for recommendation
    df = df.drop(columns=[])

    # # Remove duplicate games based on game_id and reset index again
    # df = df.drop_duplicates(subset=[]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ['game_id', 'name']
    feature_cols = [ 'max_players', 'max_playtime', 'min_age', 'min_players',
        'min_playtime', 'playing_time', 'category', 'mechanic',
        'average_rating', 'users_rated', 'category_count', 'mechanic_count',
        'has_expansion', 'len_description', 'description_sentiment']
    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = ['max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'playing_time', 
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'len_description', 'description_sentiment']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = ['has_expansion']
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = ['category', 'mechanic']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target games ID based on game name
    game_id = df.loc[(df.name == game)].game_id.values[0] 

    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # Define the number of neighbors (game list length) and use cosine similarity as the metric
    k = gamelist_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")

    # Fit the Nearest Neighbors model to the preprocessed data
    model1.fit(X_preprocessed)

    # Step 7: Extract features of the target game
    game_features = df.loc[df.game_id == game_id, feature_cols]  # Get the feature vector for the target track
    game_features_preprocessed = preprocessor.transform(game_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (songs most similar to the target game)
    distances, indices = model1.kneighbors(game_features_preprocessed)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended games
    games = df.iloc[indices[0]]  # Select tracks corresponding to the nearest neighbors
    games["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = games.columns # you can explicitly choose to return specific columns here
    games = games.loc[:, cols]  # Keep the relevant columns
    games = games.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return games.to_dict(orient="records")

In [7]:
# User input
gamelist_length = 10
game = "Catan"

response = make_recommendation(gamelist_length, game) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,len_description,description_sentiment,distance
0,13,4,120,10,3,60,Catan,120,negotiation,dice rolling,7.26569,67655,1,5,1,457,0.098867,2.220446e-16
1,30549,4,45,8,2,45,Pandemic,45,medical,action point allowance system,7.67234,62377,1,7,1,237,0.114088,1.232204e-02
2,36218,4,30,13,2,30,Dominion,30,card game,card drafting,7.69995,55930,2,3,1,289,0.130288,1.251518e-02
3,9209,5,60,8,2,30,Ticket to Ride,60,trains,hand management,7.48301,48227,2,3,1,229,0.042562,1.413778e-02
4,40692,5,80,8,2,40,Small World,80,fantasy,area control / area influence,7.35735,43144,3,4,1,250,0.084914,1.536846e-02
5,822,5,45,8,2,30,Carcassonne,45,city building,area control / area influence,7.43536,67056,3,2,1,204,-0.013889,1.717995e-02
6,68448,7,30,10,2,30,7 Wonders,30,ancient,card drafting,7.83595,51688,4,5,1,252,-0.012574,1.906051e-02
7,14996,5,60,8,2,30,Ticket to Ride: Europe,60,trains,card drafting,7.59192,35535,2,4,1,214,0.065083,1.937400e-02
8,34635,4,90,10,2,60,Stone Age,90,dice,dice rolling,7.63340,30432,2,3,1,198,0.128114,2.181270e-02
9,2651,6,120,12,2,120,Power Grid,120,economic,auction/bidding,7.94499,42036,2,2,1,215,0.051769,2.270821e-02


In [8]:
def make_recommendation_1(gamelist_length, max_players, max_playtime, min_age, min_players, min_playtime, average_rating):
    # Step 1: Load the dataset
    df = pd.read_csv("resources/ml_clean.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ['game_id', 'name']
    feature_cols = ['max_players', 'max_playtime', 'min_age', 'min_players', 'min_playtime', 'playing_time', 
                    'category', 'mechanic', 'average_rating', 'users_rated', 'category_count', 'mechanic_count',
                    'has_expansion', 'len_description', 'description_sentiment']

    # Step 3: Filter the dataset based on input parameters
    df_filtered = df[
        (df['max_players'] >= max_players) &
        (df['max_playtime'] <= max_playtime) &
        (df['min_age'] <= min_age) &
        (df['min_players'] <= min_players) &
        (df['min_playtime'] >= min_playtime) &
        (df['average_rating'] >= average_rating)
    ]

    # Step 4: Preprocess the data
    numeric_features = ['max_players', 'max_playtime', 'min_age', 'min_players', 'min_playtime', 'playing_time', 
                        'average_rating', 'users_rated', 'category_count', 'mechanic_count', 'len_description', 'description_sentiment']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    binary_features = ['has_expansion']
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('label', OrdinalEncoder())])

    categorical_features = ['category', 'mechanic']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('binary', binary_transformer, binary_features),
            ('cat', categorical_transformer, categorical_features)])

    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df_filtered.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    k = gamelist_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")
    model1.fit(X_preprocessed)

    # Step 7: Choose a random game from the filtered list to use as the target
    game_id = df_filtered.sample(1)['game_id'].values[0]
    game_features = df_filtered.loc[df_filtered.game_id == game_id, feature_cols]
    game_features_preprocessed = preprocessor.transform(game_features)

    # Step 8: Find the nearest neighbors (most similar games)
    distances, indices = model1.kneighbors(game_features_preprocessed)

    # Step 9: Retrieve the metadata of the recommended games
    games = df_filtered.iloc[indices[0]]
    games["distance"] = distances[0]

    # Step 10: Return the recommended games
    return games.to_dict(orient="records")


In [9]:
# User input
gamelist_length = 10
max_players = 4
max_playtime = 120
min_age = 5
min_players = 3
min_playtime = 60
average_rating = 5

response = make_recommendation_1(gamelist_length, max_players, max_playtime, min_age, min_players, min_playtime, average_rating) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,len_description,description_sentiment,distance
0,4550,99,60,5,3,60,1000 Blank White Cards,60,card game,deck / pool building,6.72869,483,5,3,0,387,-0.035409,0.000000
1,7771,20,90,0,2,90,En Garde!,90,adventure,paper-and-pencil,6.65743,101,3,3,0,265,0.109737,0.720924
2,33624,36,120,0,2,120,Trivial Pursuit Deluxe,120,dice,dice rolling,5.87743,113,3,2,0,117,0.272917,0.795167
3,35488,5,75,0,2,75,The Name of the Rose,75,bluffing,secret unit deployment,6.76219,1206,5,2,0,424,-0.088620,0.857293
4,40529,6,60,0,3,60,Cosmic Encounter,60,bluffing,variable player powers,7.20818,857,3,1,1,265,0.092197,0.878869
5,37165,12,60,0,2,60,"Warhammer 40,000 (fifth edition)",60,fighting,dice rolling,6.71890,967,3,3,1,129,0.028571,0.883754
6,31586,7,60,0,2,60,Obcy,60,adventure,dice rolling,6.42551,98,4,5,0,221,-0.065196,0.890595
7,39798,6,60,0,2,60,Deadlands: The Battle for Slaughter Gulch,60,adventure,area movement,5.91085,281,5,5,1,219,-0.000463,0.893136
8,34238,5,60,0,3,60,Risk: Black Ops,60,modern warfare,area control / area influence,6.95181,83,2,5,0,334,-0.064089,0.901002
9,75360,6,60,0,2,60,Texas & Pacific,60,american west,auction/bidding,6.80000,115,5,2,0,112,0.063889,0.925664
